# Import Pandas and Plotly (offline mode)

In [2]:
import pandas as pd

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# Read in Caliper Data 

Caliper key-value data formatted in JSON can be read directly with Pandas.

This particular dataset was generated using the configuration file [configs/data_centric_PEBS.config](https://github.com/LLNL/caliper-examples/blob/master/configs/data_centric_PEBS.conf)

Now, create a Pandas dataframe from the JSON data using `read_json` as follows:

In [3]:
df = pd.read_json('../datasets/lulesh_latency_analysis.json')
df

,alloc.index#libpfm.addr,alloc.label#libpfm.addr,alloc.total_size,alloc.uid,alloc.uid#libpfm.addr,cali.caliper.version,callpath.address,function,instruction.op#callpath.address,instruction.read_size#callpath.address,...,libpfm.tlb,libpfm.weight,loop,lulesh.region,mem.alloc,mem.free,pthread.id,source.file#callpath.address,source.function#callpath.address,source.line#callpath.address
0,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeElements/CalcQFo...,movsd,8,...,Hit,41.0,lulesh.cycle,10.0,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/224/776/751
1,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeNodal,movsd/leave/mov,8/0/8,...,Hit,37.0,lulesh.cycle,NaN,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/0/0/224/776/751
2,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeElements/ApplyMa...,movsd,8,...,Hit,273.0,lulesh.cycle,10.0,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/224/776/751
3,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeNodal,mov/add,0/0,...,Hit,27.0,lulesh.cycle,NaN,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/0/224/776/751
4,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeElements/ApplyMa...,jnb,0,...,Hit,37.0,lulesh.cycle,9.0,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/224/776/751
5,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeNodal/CalcForceF...,movsxd,0,...,Hit,28.0,lulesh.cycle,NaN,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/224/776/751
6,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeElements/ApplyMa...,movsd,8,...,Hit,30.0,lulesh.cycle,10.0,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/224/776/751
7,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeNodal,call,0,...,Hit,29.0,lulesh.cycle,NaN,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/224/776/751
8,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeElements/ApplyMa...,mulsd,8,...,Hit,24.0,lulesh.cycle,10.0,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/224/776/751
9,NaN,NaN,NaN,NaN,NaN,1.7.0-dev,2aaaacf5c34d/2aaaacc4fe25/2aaaab90c5e8/2aaaac8...,main/LagrangeLeapFrog/LagrangeElements/CalcLag...,mov/leave/shl/sal,8/0/0,...,Hit,29.0,lulesh.cycle,NaN,NaN,NaN,4.691709e+13,/usr/src/debug////////glibc-2.17-c758a686/misc...,__clone/start_thread/thread_wrapper/gomp_threa...,115/308/74/118/0/0/0/0/224/776/751


In [5]:
df.columns

Index(['alloc.index#libpfm.addr', 'alloc.label#libpfm.addr',
       'alloc.total_size', 'alloc.uid', 'alloc.uid#libpfm.addr',
       'cali.caliper.version', 'callpath.address', 'function',
       'instruction.op#callpath.address',
       'instruction.read_size#callpath.address',
       'instruction.write_size#callpath.address', 'iteration#lulesh.cycle',
       'libpfm.addr',
       'libpfm.counter.MEM_TRANS_RETIRED:LATENCY_ABOVE_THRESHOLD',
       'libpfm.cpu', 'libpfm.data_src', 'libpfm.event_sample_name',
       'libpfm.hit_type', 'libpfm.memory_level', 'libpfm.operation',
       'libpfm.snoop', 'libpfm.tid', 'libpfm.time', 'libpfm.tlb',
       'libpfm.weight', 'loop', 'lulesh.region', 'mem.alloc', 'mem.free',
       'pthread.id', 'source.file#callpath.address',
       'source.function#callpath.address', 'source.line#callpath.address'],
      dtype='object')

In [6]:
import os
df = df.sort_values('libpfm.time')
df['base_function'] = df['function'].transform(lambda s: os.path.basename(str(s)))

# Pandas Aggregation

The following shows how to determine the number of allocations that occurred of different sizes within different functions:

In [10]:
df_subset = df[['base_function', 'alloc.label#libpfm.addr', 'libpfm.memory_level', 'libpfm.weight']]
df_subset.groupby(['base_function', 'alloc.label#libpfm.addr', 'libpfm.memory_level']).agg(['sum','mean','count'])

libpfm.weight  \
                                                                                      sum   
base_function                   alloc.label#libpfm.addr libpfm.memory_level                 
ApplyMaterialPropertiesForElems v                       L1                           24.0   
                                                        L3                           57.0   
CalcCourantConstraintForElems   arealg                  L3                          112.0   
                                                        LFB                         419.0   
                                                        Remote Cache 1 Hops         755.0   
                                ss                      L3                          318.0   
                                                        LFB                         327.0   
                                vdov                    L3                          193.0   
                                                        LFB                          38.0   
CalcFBHourglassForceForElems    elemMass                L1                           21.0   
                                                        L3                          185.0   
                                fx                      L2                           24.0   
                                                        L3                           63.0   
                                fy                      L2                           82.0   
                                fz                      LFB                          29.0   
                                nodelist                L2                           22.0   
                                                        L3                          355.0   
                                                        LFB                        1150.0   
                                ss                      L2                           24.0   
                                                        L3                          138.0   
                                xd                      L1                           45.0   
                                                        L2                           73.0   
                                                        L3                          747.0   
                                yd                      L1                           22.0   
                                                        L2                           44.0   
                                                        L3                          558.0   
                                zd                      L2                           88.0   
                                                        L3                          601.0   
                                                        LFB                          71.0   
CalcHourglassControlForElems    nodelist                L2                           23.0   
...                                                                                   ...   
EvalEOSForElems                 ql                      L1                           21.0   
                                                        L2                           45.0   
                                                        L3                          672.0   
                                                        LFB                          23.0   
                                                        Remote Cache 1 Hops        1124.0   
                                qq                      L2                           23.0   
                                                        L3                          324.0   
                                                        LFB                         357.0   
IntegrateStressForElems         nodelist                L1                           23.0   
                                                        L2                   

# Plotly Plotting

TBD